# Functions as First-Class Objects

## Treating a Function Like an Object

In [2]:
def factorial(n):  #1
    """returns n!"""
    return 1 if n < 2 else n * factorial(n - 1)

factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__  #2

'returns n!'

In [4]:
type(factorial)

function

In [5]:
fact = factorial
fact

<function __main__.factorial(n)>

In [6]:
fact(5)

120

In [8]:
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## Higher-Order Functions

In [10]:
# sorting a list of words by length
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [11]:
def reverse(word):
    return word[::-1]

reverse('testing')

'gnitset'

In [12]:
sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### Modern Replacements for map, filter, and reduce

In [19]:
# list of factorials produced with map and filter compared to alternatives coded as list comprehensions
list(map(factorial, range(6)))  #1

[1, 1, 2, 6, 24, 120]

In [14]:
[factorial(n) for n in range(6)]  #2

[1, 1, 2, 6, 24, 120]

In [28]:
list(map(factorial, filter(lambda n: n % 2, range(6))))  #3

[1, 6, 120]

In [30]:
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

In [31]:
# Sum of integers up to 99 performed with reduce and sum
from functools import reduce  #1
from operator import add  #2

reduce(add, range(100))  #3

4950

In [32]:
sum(range(100))  #4

4950

## Anonymous Functions

In [1]:
# Sorting a list of words by their reversed spelling using lambda
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## The Nine Flavors of Callable Objects

* User-defined functions
* Built-in functions
* Built-in methods
* Methods
* Classes
* Class instances
* Generator functions
* Native coroutine functions
* Asynchronous generator functions

## User-Defined Callable Types

In [2]:
# bindocall.py: A BingoCage does one thing: picks items from a shuffled list
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)  #1
        random.shuffle(self._items)  #2
    
    def pick(self):  #3
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')  #4
    
    def __call__(self):  #5
        return self.pick()

In [3]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [4]:
bingo()

0

In [5]:
callable(bingo)

True

## From Positional to Keyword-Only Parameters

In [7]:
# tag generates HTML elements; a keyword-only argument class_ is used to pass "class" attributes
# as a workaround because class is a keyword in Python
def tag(name, *content, class_=None, **attrs):
    """Generate one or more HTML tags"""
    if class_ is not None:
        attrs['class'] = class_
    attr_pairs = (f' {attr}="{value}"' for attr, value in sorted(attrs.items()))
    
    attr_str = ''.join(attr_pairs)
    if content:
        elements = (f'<{name}{attr_str}>{c}</{name}>' for c in content)
        return '\n'.join(elements)
    else:
        return f'<{name}{attr_str} />'

In [9]:
# some of the many ways of calling the tag function
tag('br')  #1

'<br />'

In [10]:
tag('p', 'hello')  #2

'<p>hello</p>'

In [7]:
print(tag('p', 'hello', 'world'))

<p>hello</p>
<p>world</p>


In [11]:
tag('p', 'hello', id=33)  #3

'<p id="33">hello</p>'

In [12]:
print(tag('p', 'hello', 'world', class_='sidebar'))  #4

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [15]:
tag(content='testing', name='img')  #5

()


'<img content="testing" />'

In [16]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'class': 'framed'}
tag(**my_tag)  #6

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

In [17]:
def f(a, *, b):  # keyword-only arguments do not need to have a default value: they can be mandatory, like b
    return a, b

print(f(1, b=2))
print(f(1, 2))

(1, 2)


TypeError: f() takes 1 positional argument but 2 were given

### Positional-Only Parameters

In [18]:
# to define a functions requiring positional-only parameters, use / in the parameter list
# all arguments to the left of the / are positional-only. After the /, you may specify other 
# arguments, which work as usual
def divmod(a, b, /):
    return (a // b, a % b)

divmod(10, 3)

(3, 1)

## Packages for Functional Programming

### The operator Module

In [1]:
# Factorial implemented with reduce and an anonymous function
from functools import reduce

def factorial1(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

factorial1(5)

120

In [2]:
# Factorial implemented with reduce and operator.mul
from functools import reduce
from operator import mul

def factorial2(n):
    return reduce(mul, range(1, n+1))

factorial2(5)

120

In [3]:
# Demo of itemgetter to sort a list of tuples
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.68, 139.69)),
    ('Delhi NCR', 'IN', 21.935, (28.61, 77.20)),
    ('Mexico City', 'MX', 20.142, (19.43, -99.13)),
    ('New York-Newark', 'US', 20.104, (40.80, -74.02)),
    ('São Paulo', 'BR', 19.649, (-23.54, -46.63)),
]

from operator import itemgetter

for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('São Paulo', 'BR', 19.649, (-23.54, -46.63))
('Delhi NCR', 'IN', 21.935, (28.61, 77.2))
('Tokyo', 'JP', 36.933, (35.68, 139.69))
('Mexico City', 'MX', 20.142, (19.43, -99.13))
('New York-Newark', 'US', 20.104, (40.8, -74.02))


In [4]:
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'São Paulo')


In [5]:
# Demo of attrgetter to process a previously defined list of namedtuple called metro_data
from collections import namedtuple

LatLon = namedtuple('LatLon', 'lat lon')  #1
Metropolis = namedtuple('Metropolis', 'name cc pop coord')  #2
metro_areas = [Metropolis(name, cc, pop, LatLon(lat, lon))  #3
               for name, cc, pop, (lat, lon) in metro_data]
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLon(lat=35.68, lon=139.69))

In [6]:
metro_areas[0].coord.lat  #4

35.68

In [7]:
from operator import attrgetter

name_lat = attrgetter('name', 'coord.lat')  #5
for city in sorted(metro_areas, key=attrgetter('coord.lat')):  #6
    print(name_lat(city))

('São Paulo', -23.54)
('Mexico City', 19.43)
('Delhi NCR', 28.61)
('Tokyo', 35.68)
('New York-Newark', 40.8)


In [9]:
# list of functions defined in operator
import operator

print([name for name in dir(operator) if not name.startswith('_')])

['abs', 'add', 'and_', 'attrgetter', 'concat', 'contains', 'countOf', 'delitem', 'eq', 'floordiv', 'ge', 'getitem', 'gt', 'iadd', 'iand', 'iconcat', 'ifloordiv', 'ilshift', 'imatmul', 'imod', 'imul', 'index', 'indexOf', 'inv', 'invert', 'ior', 'ipow', 'irshift', 'is_', 'is_not', 'isub', 'itemgetter', 'itruediv', 'ixor', 'le', 'length_hint', 'lshift', 'lt', 'matmul', 'methodcaller', 'mod', 'mul', 'ne', 'neg', 'not_', 'or_', 'pos', 'pow', 'rshift', 'setitem', 'sub', 'truediv', 'truth', 'xor']


In [10]:
# Demo of methodcaller: second test shows the binding of extra arguments
from operator import methodcaller

s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [13]:
hyphenate = methodcaller('replace', ' ', '-')
hyphenate(s)

'The-time-has-come'

### Freezing Arguments with functools.partial

In [1]:
# Using partial to use a two-argument function where a one-argument callable is required
from operator import mul
from functools import partial

triple = partial(mul, 3)  #1
triple(7)  #2

21

In [2]:
list(map(triple, range(1, 10)))  #3

[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [4]:
# building a convenient Unicode normalizing function with partial
import unicodedata, functools

nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [5]:
s1 == s2

False

In [6]:
nfc(s1) == nfc(s2)

True

In [8]:
# Demo of partial applied to the function tag from previos example
tag  #1

<function __main__.tag(name, *content, class_=None, **attrs)>

In [9]:
from functools import partial

picture = partial(tag, 'img', class_='pic-frame')  #2
picture(src='wumpus.jpeg')  #3

'<img class="pic-frame" src="wumpus.jpeg" />'

In [10]:
picture  #4

functools.partial(<function tag at 0x7f87307ac430>, 'img', class_='pic-frame')

In [11]:
picture.func  #5

<function __main__.tag(name, *content, class_=None, **attrs)>

In [12]:
picture.args

('img',)

In [13]:
picture.keywords

{'class_': 'pic-frame'}